In [1]:
import folium #지도 
from folium.features import DivIcon #DivIcon(텍스트) 기능
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import geopandas as gpd #https://dailyheumsi.tistory.com/141참고

In [ ]:
# import requests
# import json
# import pandas as pd
# import numpy as np
# from pandas.io.json import json_normalize
# import webbrowser
# import codecs
# import folium
# from folium import plugins
# print(folium.__version__)
# import os; #pyproj_datadir = os.environ.get('PROJ_LIB[Python package 설치 경로]/pyproj/proj_base') # proj_base라는 폴더에 proj 관련 파일들을 넣어 놓았다.

In [52]:
#서울 종합 합침

seoul_crime_10man_df = pd.read_excel('./data/20191227_seoul_crime_sum_10man.xlsx')
seoul_crime_10man_df['년도'].fillna(method='ffill', inplace=True)
seoul_crime_10man_df['년도'] = seoul_crime_10man_df['년도'].astype(int)
seoul_crime_10man_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 14 columns):
년도              125 non-null int32
지역              125 non-null object
CCTV            125 non-null int64
등록외국인           125 non-null int64
한국인             125 non-null int64
합계              125 non-null int64
강간강제추행          125 non-null int64
강도              125 non-null int64
범죄합             125 non-null int64
살인              125 non-null int64
절도              125 non-null int64
폭력              125 non-null int64
10만명당 범죄 수      125 non-null int64
10만명당 CCTV 수    125 non-null int64
dtypes: int32(1), int64(12), object(1)
memory usage: 13.3+ KB


In [4]:
state_geo = './data/20191230_seoul_municipalities_geo_simple.json'
state_geo = json.load(open(state_geo, encoding='utf-8'))
#state_geo

In [43]:
state_geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

In [5]:
lo_df = pd.read_csv('./data/지역구별 기준지.txt',sep=' ',header=None)
lo_df.columns = ['지역','위도','경도']
#lo_df

In [40]:
gg_map = folium.Map(location=[37.5502, 126.982],tiles='Mapbox Bright' ,zoom_start=11) #https://python-visualization.github.io/folium/quickstart.html

folium.Choropleth(geo_data=state_geo,
                    data= seoul_crime_10man_df,
                    columns = ['지역', '등록외국인'],
                    fill_color='OrRd', fill_opacity=0.8, line_opacity=0.7, #fill_opacity - 투명도, 높을수록 짙어짐   line_opacity - 선 굵기, 높을수록 굵어짐
                    legend_name = '등록외국인 수',
                    #threshold_scale=[10000, 100000, 300000, 500000, 700000],
                    key_on = 'feature.properties.name').add_to(gg_map)


#텍스트 표시
for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
    folium.map.Marker(
        [at, lng-0.01],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 7pt">%s</div>' % name,
            )
        ).add_to(gg_map)


gg_map.save('./data/20191230_seoul_map_등록외국인수.html')
gg_map

In [24]:

g_map = folium.Map(location=[37.5502, 126.982], tiles='Mapbox Bright' , zoom_start=11)


#지도표시
folium.Choropleth(geo_data=state_geo,
                    data= seoul_crime_10man_df[year_2018],
                    columns = ['지역', '범죄합'],
                    fill_color='OrRd', fill_opacity=1.0, line_opacity=0.7, #fill_opacity - 투명도, 높을수록 짙어짐   line_opacity - 선 굵기, 높을수록 굵어짐
                    legend_name = '모든범죄합계',
                    #threshold_scale=[10000, 100000, 300000, 500000, 700000],
                    key_on = 'feature.properties.name').add_to(g_map)

#원 마커 표시
# for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
#     #print(name, at ,lng)
#     #folium.Marker([at, lng], popup=name).add_to(seoul_map) #Document에 다양한 종류의 마커가 있다
#     folium.CircleMarker([at,lng], radius=10 , color='black', fill=True , fill_color='green',
#                        fill_opacity=0.7, popup=name).add_to(g_map)

#원 표시
# for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
#     #print(name, at ,lng)
#     #folium.Marker([at, lng], popup=name).add_to(seoul_map) #Document에 다양한 종류의 마커가 있다
#     folium.Circle([at, lng], 1500, fill=True).add_child(folium.Popup(name)).add_to(g_map)


#텍스트 표시
for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
    folium.map.Marker(
        [at, lng-0.01],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 7pt">%s</div>' % name,
            )
        ).add_to(g_map)


g_map.save('./data/20191230_seoul_map_모든범죄합계.html')
g_map

In [8]:
seoul_crime_10man_df[seoul_crime_10man_df['년도'] == 2014].sort_values(by=['10만명당 CCTV 수'], axis=0, ascending=False).head(5)

,년도,지역,CCTV,등록외국인,한국인,합계,강간강제추행,강도,범죄합,살인,절도,폭력,10만명당 범죄 수,10만명당 CCTV 수
10,2014,동대문구,1326,12142,363567,375709,141,13,4363,12,1809,2388,1161,352
24,2014,중랑구,770,4455,418475,422930,163,14,5353,11,2390,2775,1265,182
11,2014,동작구,503,10715,411275,421990,203,9,3869,2,1787,1868,916,119
4,2014,관악구,487,17085,517783,534868,361,25,6781,7,3029,3359,1267,91
22,2014,종로구,132,7799,159551,167350,226,12,5021,3,2272,2508,3000,78


In [9]:
seoul_crime_10man_df[seoul_crime_10man_df['년도'] == 2014].sort_values(by=['10만명당 범죄 수'], axis=0, ascending=False).head(5)

,년도,지역,CCTV,등록외국인,한국인,합계,강간강제추행,강도,범죄합,살인,절도,폭력,10만명당 범죄 수,10만명당 CCTV 수
23,2014,중구,80,7526,129940,137466,221,13,5231,6,2576,2415,3805,58
22,2014,종로구,132,7799,159551,167350,226,12,5021,3,2272,2508,3000,78
19,2014,영등포구,217,34905,385993,420898,250,10,7003,14,3080,3649,1663,51
5,2014,광진구,87,12692,368325,381017,249,16,6268,8,3167,2828,1645,22
0,2014,강남구,430,5399,565302,570701,512,37,8851,13,3895,4394,1550,75


In [10]:
year_2018 = seoul_crime_10man_df['년도'] == 2018

In [53]:
#살인, 강간강제추행 - 살인강간 
seoul_crime_10man_df['살인강간'] =  seoul_crime_10man_df['살인'] + seoul_crime_10man_df['강간강제추행']
#강도, 절도, 폭력 - 강절폭
seoul_crime_10man_df['강절폭'] =  seoul_crime_10man_df['강도'] + seoul_crime_10man_df['절도'] +seoul_crime_10man_df['폭력']
seoul_crime_10man_df[year_2018]

,년도,지역,CCTV,등록외국인,한국인,합계,강간강제추행,강도,범죄합,살인,절도,폭력,10만명당 범죄 수,10만명당 CCTV 수,살인강간,강절폭
100,2018,강남구,2766,4889,552976,557865,600,14,7513,10,3004,3885,1346,495,610,6903
101,2018,강동구,1055,4198,434027,438225,170,7,3919,3,1515,2224,894,240,173,3746
102,2018,강북구,793,3649,323862,327511,188,7,3437,8,1112,2122,1049,242,196,3241
103,2018,강서구,1125,6665,601696,608361,269,6,4629,11,1724,2619,760,184,280,4349
104,2018,관악구,3080,17847,504445,522292,352,10,5026,9,2012,2643,962,589,361,4665
105,2018,광진구,986,14522,357149,371671,229,2,3915,7,1769,1908,1053,265,236,3679
106,2018,구로구,1807,32027,408369,440396,250,9,4810,8,1717,2826,1092,410,258,4552
107,2018,금천구,1348,19106,234238,253344,200,7,3293,6,1168,1912,1299,532,206,3087
108,2018,노원구,1105,3901,551902,555803,194,5,4007,5,1422,2381,720,198,199,3808
109,2018,도봉구,587,2180,342861,345041,113,5,2249,1,781,1349,651,170,114,2135


In [18]:
#살인강간
gg_map = folium.Map(location=[37.5502, 126.982],tiles='Mapbox Bright' ,zoom_start=11) #https://python-visualization.github.io/folium/quickstart.html

folium.Choropleth(geo_data=state_geo,
                    data= seoul_crime_10man_df[year_2018],
                    columns = ['지역', '살인강간'],
                    fill_color='OrRd', fill_opacity=0.8, line_opacity=0.7, #fill_opacity - 투명도, 높을수록 짙어짐   line_opacity - 선 굵기, 높을수록 굵어짐
                    legend_name = '살인강간',
                    #threshold_scale=[10000, 100000, 300000, 500000, 700000],
                    key_on = 'feature.properties.name').add_to(gg_map)


#텍스트 표시
for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
    folium.map.Marker(
        [at, lng-0.01],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 7pt">%s</div>' % name,
            )
        ).add_to(gg_map)


gg_map.save('./data/20191231_seoul_map_살인강간.html')
gg_map

In [121]:
#강절폭
gg_map = folium.Map(location=[37.5502, 126.982],tiles='Mapbox Bright' ,zoom_start=11) #https://python-visualization.github.io/folium/quickstart.html

folium.Choropleth(geo_data=state_geo,
                    data= seoul_crime_10man_df[year_2018],
                    columns = ['지역', '강절폭'],
                    fill_color='OrRd', fill_opacity=0.8, line_opacity=0.7, #fill_opacity - 투명도, 높을수록 짙어짐   line_opacity - 선 굵기, 높을수록 굵어짐
                    legend_name = '강절폭',
                    #threshold_scale=[2249, 3126, 4004, 4881, 5758, 6636, 7513],
                    key_on = 'feature.properties.name').add_to(gg_map)


#텍스트 표시
for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
    folium.map.Marker(
        [at, lng-0.01],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 7pt">%s</div>' % name,
            )
        ).add_to(gg_map)


gg_map.save('./data/20191231_seoul_map_강도절도폭력.html')
gg_map

In [56]:
seoul_crime_10man_df_2018 = seoul_crime_10man_df[year_2018]
seoul_crime_10man_df_2018.reset_index(drop=True, inplace=True)
seoul_crime_10man_df_2018

,년도,지역,CCTV,등록외국인,한국인,합계,강간강제추행,강도,범죄합,살인,절도,폭력,10만명당 범죄 수,10만명당 CCTV 수,살인강간,강절폭
0,2018,강남구,2766,4889,552976,557865,600,14,7513,10,3004,3885,1346,495,610,6903
1,2018,강동구,1055,4198,434027,438225,170,7,3919,3,1515,2224,894,240,173,3746
2,2018,강북구,793,3649,323862,327511,188,7,3437,8,1112,2122,1049,242,196,3241
3,2018,강서구,1125,6665,601696,608361,269,6,4629,11,1724,2619,760,184,280,4349
4,2018,관악구,3080,17847,504445,522292,352,10,5026,9,2012,2643,962,589,361,4665
5,2018,광진구,986,14522,357149,371671,229,2,3915,7,1769,1908,1053,265,236,3679
6,2018,구로구,1807,32027,408369,440396,250,9,4810,8,1717,2826,1092,410,258,4552
7,2018,금천구,1348,19106,234238,253344,200,7,3293,6,1168,1912,1299,532,206,3087
8,2018,노원구,1105,3901,551902,555803,194,5,4007,5,1422,2381,720,198,199,3808
9,2018,도봉구,587,2180,342861,345041,113,5,2249,1,781,1349,651,170,114,2135


In [57]:
#범죄 합
display(seoul_crime_10man_df_2018[['지역','범죄합']].sort_values(by=['범죄합'], axis=0, ascending=False).head(5))

#살인강간
display(seoul_crime_10man_df_2018[['지역','살인강간']].sort_values(by=['살인강간'], axis=0, ascending=False).head(5))

#강절폭
display(seoul_crime_10man_df_2018[['지역','강절폭']].sort_values(by=['강절폭'], axis=0, ascending=False).head(5))

#등록외국인수
display(seoul_crime_10man_df_2018[['지역','등록외국인']].sort_values(by=['등록외국인'], axis=0, ascending=False).head(5))

,지역,범죄합
0,강남구,7513
19,영등포구,5840
17,송파구,5807
12,마포구,5172
4,관악구,5026


,지역,살인강간
0,강남구,610
12,마포구,479
14,서초구,475
19,영등포구,408
4,관악구,361


,지역,강절폭
0,강남구,6903
17,송파구,5493
19,영등포구,5432
12,마포구,4693
4,관악구,4665


,지역,등록외국인
19,영등포구,34860
6,구로구,32027
7,금천구,19106
4,관악구,17847
10,동대문구,15654


In [148]:
food_df = pd.read_csv('./data/20191231_seoul_식품위생업_데이터_2014_to_2018.csv')
f_2018 = food_df['년도'] == 2018
food_df = food_df[f_2018]
food_df

,년도,지역,단란주점,유흥주점
100,2018,종로구,130,210
101,2018,중구,92,233
102,2018,용산구,108,29
103,2018,성동구,41,23
104,2018,광진구,86,18
105,2018,동대문구,91,71
106,2018,중랑구,103,24
107,2018,성북구,65,8
108,2018,강북구,126,74
109,2018,도봉구,55,23


In [149]:
food_df.sort_values(by=['지역'], axis=0, ascending=False,inplace=True)
food_df.reset_index(drop=True,inplace=True)
seoul_crime_10man_df_2018.sort_values(by=['지역'], axis=0, ascending=False,inplace=True)
seoul_crime_10man_df_2018.reset_index(drop=True,inplace=True)

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1]:
x = 100000 / 411552
x

0.24298266075732836

In [150]:
seoul_crime_10man_df_2018

,년도,지역,CCTV,등록외국인,한국인,합계,강간강제추행,강도,범죄합,살인,절도,폭력,10만명당 범죄 수,10만명당 CCTV 수,살인강간,강절폭,단란주점,유흥주점,단란유흥
0,2018,중랑구,1068,4684,406868,411552,174,3,4288,4,1526,2581,1041,259,178,4110,103,24,127
1,2018,중구,1240,9057,126082,135139,207,11,4030,2,1855,1955,2982,917,209,3821,92,233,325
2,2018,종로구,857,9799,154549,164348,236,3,3690,6,1483,1962,2245,521,242,3448,130,210,340
3,2018,은평구,1415,4351,485902,490253,188,1,3590,8,1278,2115,732,288,196,3394,137,82,219
4,2018,용산구,431,15502,229909,245411,331,3,3411,1,1096,1980,1389,175,332,3079,108,29,137
5,2018,영등포구,2036,34860,369128,403988,391,8,5840,17,2292,3132,1445,503,408,5432,208,159,367
6,2018,양천구,1657,3866,469221,473087,120,4,3515,2,1384,2005,742,350,122,3393,73,5,78
7,2018,송파구,799,6712,665282,671994,309,6,5807,5,2352,3135,864,118,314,5493,161,99,260
8,2018,성북구,1836,11676,442226,453902,183,1,3042,7,1164,1687,670,404,190,2852,65,8,73
9,2018,성동구,1787,8019,306532,314551,126,5,2457,2,999,1325,781,568,128,2329,41,23,64


In [152]:
for i in range(len(food_df)):
    #print(i)
    seoul_crime_10man_df_2018.loc[i, '단란주점'] = food_df.loc[i, '단란주점']
    seoul_crime_10man_df_2018.loc[i, '유흥주점'] = food_df.loc[i, '유흥주점']
    
seoul_crime_10man_df_2018[['단란주점', '유흥주점']] = seoul_crime_10man_df_2018[['단란주점', '유흥주점']].astype(int)

C:\Users\student\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\student\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [153]:
seoul_crime_10man_df_2018['단란유흥'] =  seoul_crime_10man_df_2018['단란주점'] + seoul_crime_10man_df_2018['유흥주점']

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [154]:
seoul_crime_10man_df_2018

,년도,지역,CCTV,등록외국인,한국인,합계,강간강제추행,강도,범죄합,살인,절도,폭력,10만명당 범죄 수,10만명당 CCTV 수,살인강간,강절폭,단란주점,유흥주점,단란유흥
0,2018,중랑구,1068,4684,406868,411552,174,3,4288,4,1526,2581,1041,259,178,4110,103,24,127
1,2018,중구,1240,9057,126082,135139,207,11,4030,2,1855,1955,2982,917,209,3821,92,233,325
2,2018,종로구,857,9799,154549,164348,236,3,3690,6,1483,1962,2245,521,242,3448,130,210,340
3,2018,은평구,1415,4351,485902,490253,188,1,3590,8,1278,2115,732,288,196,3394,137,82,219
4,2018,용산구,431,15502,229909,245411,331,3,3411,1,1096,1980,1389,175,332,3079,108,29,137
5,2018,영등포구,2036,34860,369128,403988,391,8,5840,17,2292,3132,1445,503,408,5432,208,159,367
6,2018,양천구,1657,3866,469221,473087,120,4,3515,2,1384,2005,742,350,122,3393,73,5,78
7,2018,송파구,799,6712,665282,671994,309,6,5807,5,2352,3135,864,118,314,5493,161,99,260
8,2018,성북구,1836,11676,442226,453902,183,1,3042,7,1164,1687,670,404,190,2852,65,8,73
9,2018,성동구,1787,8019,306532,314551,126,5,2457,2,999,1325,781,568,128,2329,41,23,64


In [141]:
#단란유흥
gg_map = folium.Map(location=[37.5502, 126.982],tiles='Mapbox Bright' ,zoom_start=11) #https://python-visualization.github.io/folium/quickstart.html

folium.Choropleth(geo_data=state_geo,
                    data= seoul_crime_10man_df_2018,
                    columns = ['지역', '단란유흥'],
                    fill_color='PuRd', fill_opacity=0.8, line_opacity=0.7, #fill_opacity - 투명도, 높을수록 짙어짐   line_opacity - 선 굵기, 높을수록 굵어짐
                    legend_name = '단란유흥',
                    #threshold_scale=[2249, 3126, 4004, 4881, 5758, 6636, 7513],
                    key_on = 'feature.properties.name').add_to(gg_map)


#텍스트 표시
for name, at, lng in zip(lo_df.지역, lo_df.위도, lo_df.경도):
    folium.map.Marker(
        [at, lng-0.01],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 7pt">%s</div>' % name,
            )
        ).add_to(gg_map)


gg_map.save('./data/20191231_seoul_map_단란유흥.html')
gg_map